In [83]:
import time

from ib.opt import message
import numpy as np
import pandas as pd

import contracts
from settings import TWS_CONNECTION, company

In [49]:
class GetOptionsPrices():
    contract = None
    right = None
    prices_put = pd.DataFrame()
    prices_call = pd.DataFrame()
    prices_msg = pd.DataFrame()
   
    def __init__(self, company, expiration, strike):
        self.company = company
        self.expiration = expiration
        self.strike = strike

    def get_prices(self):
        TWS_CONNECTION.register(self.msg_handler, message.historicalData)
        TWS_CONNECTION.register(self.error_handler, message.Error)
#         TWS_CONNECTION.registerAll(print)
        self.get_put_prices()
        prices_put = self.prices_msg.set_index('Datetime')
        self.prices_msg = pd.DataFrame()
        self.get_call_prices()
        prices_call = self.prices_msg.set_index('Datetime')
        # Merge PUT & CALL into one DateFrame
        return pd.concat([prices_call, prices_put], axis=1, sort=False)
       
    def get_put_prices(self):
        TWS_CONNECTION.connect()
        self.right = 'P' # for the column name
        self.contract = contracts.PutOption(
            self.company,
            self.expiration,
            self.strike
        )
        self.request_prices()
        time.sleep(2)
        TWS_CONNECTION.disconnect()

    def get_call_prices(self):
        TWS_CONNECTION.connect()
        self.right = 'C' # for the column name
        self.contract = contracts.CallOption(
            self.company,
            self.expiration,
            self.strike
        )
        self.request_prices()
        time.sleep(2)
        TWS_CONNECTION.disconnect()
 
    def request_prices(self):
        TWS_CONNECTION.reqHistoricalData(
            1,
            self.contract,
            '',
            '1 W',
            '30 mins',
            "TRADES",
            1,
            1
        )
        
    def msg_handler(self, msg):
        """
        Function get message from TWS and put it into pandas DataFrame object
        :param msg: message from TWS
        :return: dataframe w/ historical prices
        """
        new_row = pd.DataFrame(
            {'Datetime': msg.date,
            f'{self.right}-{self.expiration}-{self.strike}': msg.close},
            index=[1])
        self.prices_msg = self.prices_msg.append(new_row, ignore_index=True)
    
    def error_handler(self, msg):
        print(type(msg), msg)        

In [48]:
a = {
	'date': '20191127  21:30:00',
	'P-2019-12-06': 0.37,
}
b = {
	'date': '20191128  21:30:00',
	'P-2019-12-06': 1.37,
}
c = {
	'date': '20191128  21:30:00',
	'C-2019-12-06': 1.37,
}
test = pd.DataFrame()
a = pd.DataFrame(a, index = [0]).set_index('date')
b = pd.DataFrame(b, index = [0]).set_index('date')
test = test.append(a)
test = test.append(b)
# print(test)
c = pd.DataFrame(c, index = [0]).set_index('date')
# print(test)
# print(c)
# test = test.merge(c)#, left_index=True, right_index=True)
test = pd.concat([test, c], axis=1, sort=False)
print(test)

                    P-2019-12-06  C-2019-12-06
20191127  21:30:00          0.37           NaN
20191128  21:30:00          1.37          1.37


In [78]:
prices2 = GetOptionsPrices(
    company,
    '20200110',
    450
).get_prices()
prices2

<class 'ib.opt.message.Error'> <error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use. Retry with a unique client id.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use. Retry with a unique client id.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use. Retry with a unique client id.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use. Retry with a unique client id.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use. Retry with a unique client id.>
<error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use. Retry with a unique client id.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=326, errorMsg=Unable connect as the client id is already in use.

KeyError: 'Datetime'

In [36]:
prices = GetOptionsPrices(
    company,
    '20191206',
    300
).get_prices()

<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection

In [37]:
prices

,C-20191206-300,P-20191206-300
Datetime,,
20191125 17:30:00,43.05,0.44
20191125 18:00:00,43.05,0.46
20191125 18:30:00,38.95,0.53
20191125 19:00:00,38.95,0.54
20191125 19:30:00,38.95,0.52
20191125 20:00:00,38.95,0.46
20191125 20:30:00,38.95,0.48
20191125 21:00:00,38.95,0.50
20191125 21:30:00,38.95,0.50


In [81]:
# Look for all strikes and expirations
from ib.ext.ContractDetails import ContractDetails
from ib.ext.EClientSocket import EClientSocket


def get_option_chain():
    contract = contracts.CallOption(
        company,
        None,
        None
    )
    options = []
    wait = 0.1

    def msg_stopper(msg):
        wait = False
        
    def msg_handler(msg):
        options.append({
            'strike': msg.contractDetails.m_summary.m_strike,
            'expiration': msg.contractDetails.m_summary.m_expiry
        })

    TWS_CONNECTION.registerAll(print)
    TWS_CONNECTION.register(msg_handler, 'ContractDetails')
    TWS_CONNECTION.register(msg_stopper, 'ContractDetailsEnd')
    TWS_CONNECTION.connect()
    EClientSocket.reqContractDetails(TWS_CONNECTION, 1, contract)
    while wait and wait < 10:
        time.sleep(0.1)
        wait += 0.1
    TWS_CONNECTION.disconnect()
    return options

In [84]:
options = get_option_chain()

<class 'ib.opt.message.Error'> <error id=-1, errorCode=502, errorMsg=Couldn't connect to TWS.  Confirm that "Enable ActiveX and Socket Clients" is enabled on the TWS "Configure->API" menu.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=502, errorMsg=Couldn't connect to TWS.  Confirm that "Enable ActiveX and Socket Clients" is enabled on the TWS "Configure->API" menu.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=502, errorMsg=Couldn't connect to TWS.  Confirm that "Enable ActiveX and Socket Clients" is enabled on the TWS "Configure->API" menu.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=502, errorMsg=Couldn't connect to TWS.  Confirm that "Enable ActiveX and Socket Clients" is enabled on the TWS "Configure->API" menu.>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=502, errorMsg=Couldn't connect to TWS.  Confirm that "Enable ActiveX and Socket Clients" is enabled on the TWS "Configure->API" menu.>
<error id=-1, errorCode=502, errorMsg=Couldn't con

In [66]:
from ib.opt import message
type(msgs[-1]) == message.contractDetailsEnd


True

In [50]:
! git commit -a -m 'In work'
! git push

[version_2 2910f6c] In work
 2 files changed, 730 insertions(+), 876 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 2.12 KiB | 1.06 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/SVGalich/tws_trader.git
   99e08d3..2910f6c  version_2 -> version_2
